# <span style="font-width:bold; font-size: 3rem; color:#1EB182;"><img src="images/icon102.png" width="38px"></img> **Hopsworks Feature Store** </span><span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Training Data & Feature views</span>

<span style="font-width:bold; font-size: 1.4rem;">This is the second part of the quick start series of tutorials about Hopsworks Feature Store. This notebook explains how to read from a feature group and create training dataset within the feature store</span>

## **🗒️ In this notebook we will see how to create a training dataset from the feature groups:** 
1. **Select the features** we want to train our model on,
2. **How the features should be preprocessed,**
3. **Create a dataset split** for training and validation data.

![tutorial-flow](images/02_training-dataset.png) 

In [1]:
import hsfs

conn = hsfs.connection()
fs = conn.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.


## <span style="color:#ff5f27;"> 🔪 Feature Selection </span>

We start by selecting all the features we want to include for model training/inference.

In [2]:
# Load feature groups.
trans_fg = fs.get_feature_group('transactions', version=1)
window_aggs_fg = fs.get_feature_group('transactions_4h_aggs', version=1)

# Select features for training data.
ds_query = trans_fg.select(["fraud_label", "amount", "age_at_transaction", "days_until_card_expires", "loc_delta"])\
    .join(window_aggs_fg.select_except(["cc_num"]), on="cc_num")\

ds_query.show(5)

2022-06-16 15:34:55,871 INFO: USE `online_fraud_featurestore`
2022-06-16 15:34:56,798 INFO: WITH right_fg0 AS (SELECT *
FROM (SELECT `fg1`.`fraud_label` `fraud_label`, `fg1`.`amount` `amount`, `fg1`.`age_at_transaction` `age_at_transaction`, `fg1`.`days_until_card_expires` `days_until_card_expires`, `fg1`.`loc_delta` `loc_delta`, `fg1`.`cc_num` `join_pk_cc_num`, `fg1`.`datetime` `join_evt_datetime`, `fg0`.`loc_delta_mavg` `loc_delta_mavg`, RANK() OVER (PARTITION BY `fg1`.`cc_num`, `fg1`.`datetime` ORDER BY `fg0`.`datetime` DESC) pit_rank_hopsworks
FROM `online_fraud_featurestore`.`transactions_1` `fg1`
INNER JOIN `online_fraud_featurestore`.`transactions_4h_aggs_1` `fg0` ON `fg1`.`cc_num` = `fg0`.`cc_num` AND `fg1`.`datetime` >= `fg0`.`datetime`) NA
WHERE `pit_rank_hopsworks` = 1) (SELECT `right_fg0`.`fraud_label` `fraud_label`, `right_fg0`.`amount` `amount`, `right_fg0`.`age_at_transaction` `age_at_transaction`, `right_fg0`.`days_until_card_expires` `days_until_card_expires`, `right_f

,fraud_label,amount,age_at_transaction,days_until_card_expires,loc_delta,loc_delta_mavg
0,0,800.35,45.213897,514.963148,0.000000,0.000000
1,0,7.73,45.215266,514.463148,0.000095,0.000095
2,0,44.76,45.216635,513.963148,0.000173,0.000173
3,0,958.34,45.218004,513.463148,0.000185,0.000185
4,0,89.00,45.219373,512.963148,0.000103,0.000103


Recall that we computed the features in `transactions_4h_aggs` using 4-hour aggregates. If we had created multiple feature groups with identical schema for different window lengths, and wanted to include them in the join we would need to include a prefix argument in the join to avoid feature name clash. See the [documentation](https://docs.hopsworks.ai/feature-store-api/latest/generated/api/query_api/#join) for more details.

v🤖 Transformation Functions </span>

We will preprocess our data using *min-max scaling* on numerical features and *label encoding* on categorical features. To do this we simply define a mapping between our features and transformation functions. This ensures that transformation functions such as *min-max scaling* are fitted only on the training data (and not the validation/test data), which ensures that there is no data leakage.

In [3]:
# Load transformation functions.
min_max_scaler = fs.get_transformation_function(name="min_max_scaler")
label_encoder = fs.get_transformation_function(name="label_encoder")

# Map features to transformations.
transformation_functions = {
    "amount": min_max_scaler,
    "loc_delta": min_max_scaler,
    "loc_delta_mavg": min_max_scaler,
    "age_at_transaction": min_max_scaler,
    "days_until_card_expires": min_max_scaler,
}

## <span style="color:#ff5f27;"> ⚙️ Feature View Creation </span>

The Feature Views allows schema in form of a query with filters, define a model target feature/label and additional transformation functions.
In order to create a Feature View we may use `fs.create_feature_view()`

In [4]:
feature_view = fs.create_feature_view(
    name='transactions_view',
    query=ds_query,
    labels=["fraud_label"],
    transformation_functions=transformation_functions
)

Feature view created successfully, explore it at https://hopsworks0.logicalclocks.com/p/119/fs/67/fv/transactions_view/version/1


To view and explore data in the feature view we can retrieve batch data using `get_batch_data()` method 

## <span style="color:#ff5f27;"> 🏋️ Training Dataset Creation</span>

In Hopsworks training data is a query where the projection (set of features) is determined by the parent FeatureView with an optional snapshot on disk of the data returned by the query.

**Training Dataset  may contain splits such as:** 
* Training set - the subset of training data used to train a model.
* Validation set - the subset of training data used to evaluate hparams when training a model
* Test set - the holdout subset of training data used to evaluate a mode

Training dataset is created using `fs.create_training_dataset()` method.

**From feature view APIs we can also create training datasts based on even time filters specifing `start_time` and `end_time`** 



In [5]:
from datetime import datetime
date_format = "%Y-%m-%d %H:%M:%S"

In [6]:
# Create training datasets based event time filter
start_time = int(float(datetime.strptime("2022-01-01 00:00:01", date_format).timestamp()) * 1000)
end_time = int(float(datetime.strptime("2022-02-28 23:59:59", date_format).timestamp()) * 1000)

td_jan_feb_version, td_job = feature_view.create_training_dataset(
        start_time = start_time,
        end_time = end_time,
        description = 'transactions_dataset_jan_feb',
        data_format = "csv",
        coalesce = True,
        write_options = {'wait_for_job': True},
    )

Training dataset job started successfully, you can follow the progress at https://hopsworks0.logicalclocks.com/p/119/jobs/named/transactions_view_1_1_create_fv_td_16062022153623/executions


In [7]:
start_time = int(float(datetime.strptime("2022-03-01 00:00:01", date_format).timestamp()) * 1000)
end_time = int(float(datetime.strptime("2022-03-31 23:59:59", date_format).timestamp()) * 1000)

td_mar_version, td_job = feature_view.create_training_dataset(
        start_time = start_time,
        end_time = end_time,
        description = 'transactions_dataset_mar',
        data_format = "csv",
        coalesce = True,
        write_options = {'wait_for_job': True},
 )


Training dataset job started successfully, you can follow the progress at https://hopsworks0.logicalclocks.com/p/119/jobs/named/transactions_view_1_2_create_fv_td_16062022154029/executions


## <span style="color:#ff5f27;"> 🪝 Training Dataset retreival </span>

To retrieve training data from storage (already materialised) or from feature groups direcly we can use `get_training_dataset_splits` or `get_training_dataset` methods. If version is not provided or provided version has not already existed, it creates a new version of training data according to given arguments and returns a dataframe. If version is provided and has already existed, it reads training data from storage or feature groups and returns a dataframe. If split is provided, it reads the specific split.

In [8]:
train_jan_feb_x, train_jan_feb_y = feature_view.get_training_data(td_jan_feb_version)

In [9]:
train_mar_x, train_mar_y = feature_view.get_training_data(td_mar_version)

In [11]:
train_jan_feb_x

,amount,age_at_transaction,days_until_card_expires,loc_delta,loc_delta_mavg
0,0.002478,0.451919,0.873791,0.106689,0.108080
1,0.000247,0.628477,0.512349,0.045635,0.046230
2,0.191015,0.033169,0.853372,0.236723,0.239810
3,0.002814,0.387324,0.307864,0.099704,0.156792
4,0.001679,0.632300,0.813488,0.000029,0.000030
...,...,...,...,...,...
38978,0.003110,0.665616,0.738367,0.000054,0.000055
38979,0.001577,0.578103,0.563284,0.101315,0.102636
38980,0.001344,0.574697,0.763167,0.467712,0.348457
38981,0.000972,0.614414,0.075533,0.000055,0.000055


In [12]:
train_jan_feb_y

,fraud_label
0,0
1,0
2,0
3,0
4,0
...,...
38978,0
38979,0
38980,0
38981,0


In [13]:
train_mar_x

,amount,age_at_transaction,days_until_card_expires,loc_delta,loc_delta_mavg
0,0.010817,0.607926,0.013914,0.315866,0.315878
1,0.015404,0.009279,0.402535,0.503823,0.503823
2,0.000643,0.047399,0.420320,0.255525,0.255525
3,0.002438,0.375528,0.048967,0.036604,0.036604
4,0.032730,0.452827,0.661048,0.348952,0.348952
...,...,...,...,...,...
11877,0.002475,0.343570,0.198044,0.166585,0.083338
11878,0.024626,0.054586,0.091435,0.212351,0.212351
11879,0.024172,0.522471,0.428013,0.139787,0.139787
11880,0.076640,0.665787,0.744032,0.450916,0.225500


In [14]:
train_mar_y

,fraud_label
0,0
1,0
2,0
3,0
4,0
...,...
11877,0
11878,0
11879,0
11880,0


## <span style="color:#ff5f27;">⏭️ **Next:** Part 03 </span>

In the following notebook, we will train a model on the dataset we created in this notebook and have quick overview of the lineage.